In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 https://cloud.r-project.

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-05-04 04:44:49--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.81MB/s    in 0.2s    

2022-05-04 04:44:50 (5.81 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Video_Download_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12190288|R3FU16928EP5TC|B00AYB1482|     668895143|Enlightened: Seas...|Digital_Video_Dow...|          5|            0|          0|   N|                Y|I loved it and I ...|I loved it and I ...| 2015-08-31|
|         US|   30549954|R1IZHHS1MH3AQ4|B00KQD28OM|     246219280|             Vicious|Digital_Video_Dow

In [5]:
from pyspark.sql.functions import to_date
from pyspark.sql.functions import col,isnan, when, count
null_count_df = df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns])
null_count_df.show()

+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+
|marketplace|customer_id|review_id|product_id|product_parent|product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|review_headline|review_body|review_date|
+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+
|          0|          0|        0|         0|             0|            0|               0|          0|            0|          0|   0|                0|              4|        613|        548|
+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+



In [6]:
# Drop Nulls
cleaned_df = df.dropna()
cleaned_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12190288|R3FU16928EP5TC|B00AYB1482|     668895143|Enlightened: Seas...|Digital_Video_Dow...|          5|            0|          0|   N|                Y|I loved it and I ...|I loved it and I ...| 2015-08-31|
|         US|   30549954|R1IZHHS1MH3AQ4|B00KQD28OM|     246219280|             Vicious|Digital_Video_Dow

In [9]:
# Cleaned DataFrame total unique reviews.
from pyspark.sql.functions import countDistinct
cleaned_df.select(countDistinct("review_id")).show()

+-------------------------+
|count(DISTINCT review_id)|
+-------------------------+
|                  4056518|
+-------------------------+



In [10]:
# Create DF where the rows are equal to or greater than 20 to pick reviews that are more likely to be
# helpful and to avoid having division by zero errors later on.
review_analysis_df = cleaned_df.select(["review_id", "product_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
votes20_df = review_analysis_df.filter("total_votes >= 20")
votes20_df.show()

+--------------+----------+-----------+-------------+-----------+----+-----------------+
|     review_id|product_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+----------+-----------+-------------+-----------+----+-----------------+
|R1CY3APKBOVUYU|B00VO8D13K|          1|            9|         33|   N|                Y|
|R2LTI28AN2281A|B00YOSAJBO|          1|            0|         22|   N|                Y|
|R3PENXZQNYN29L|B00XOX9QCS|          1|            1|         26|   N|                Y|
| R8IGXNWIBN7ZP|B007BVMVDU|          1|            6|         20|   N|                Y|
|R2P8B83Q61L4HS|B00XOXDXV8|          1|            0|         22|   N|                Y|
| RZM83AGJAUZMA|B0142KIQYA|          1|           32|         41|   N|                Y|
|R257YW4XQLDH1M|B006W0QOF2|          1|            6|         20|   N|                Y|
|R12MYB9QM2YN8B|B00OLI95R6|          1|           19|         23|   N|                N|
| RGCY6WG37F810|B00ZN

In [13]:
# Filter rows where number of helpful_votes divided by total_votes is equal to or greater than 50%
helpful_df = votes20_df.filter(votes20_df["helpful_votes"]/votes20_df["total_votes"] >= 0.5)
helpful_df.show()

+--------------+----------+-----------+-------------+-----------+----+-----------------+
|     review_id|product_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+----------+-----------+-------------+-----------+----+-----------------+
| RZM83AGJAUZMA|B0142KIQYA|          1|           32|         41|   N|                Y|
|R12MYB9QM2YN8B|B00OLI95R6|          1|           19|         23|   N|                N|
| RGCY6WG37F810|B00ZN31MK6|          4|           24|         27|   N|                N|
|R2UH0VN09DDQLA|B0143LIXI2|          3|           23|         30|   N|                N|
|R1E4W9ROP10WT3|B013QMAZ80|          5|           19|         22|   N|                N|
| RF2BLXZTUOL0M|B013WF1Z4O|          5|           23|         27|   N|                N|
| REMDS9V4J58OR|B00X5FYC7Y|          4|           25|         29|   N|                Y|
|R34CAWTWR5O2VN|B0142KIQYA|          1|           18|         20|   N|                Y|
|R2ISFKK57JF02V|B00R6

In [14]:
# Filter rows where a review was written as part of the Vine paid program
vine_paid = helpful_df.filter(helpful_df["vine"] == "Y")
vine_paid.show()

+---------+----------+-----------+-------------+-----------+----+-----------------+
|review_id|product_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+---------+----------+-----------+-------------+-----------+----+-----------------+
+---------+----------+-----------+-------------+-----------+----+-----------------+



In [15]:
# Filter rows where a review was written that was not part of the Vine paid program
vine_not_paid = helpful_df.filter(helpful_df["vine"] == "N")
vine_not_paid.show()

+--------------+----------+-----------+-------------+-----------+----+-----------------+
|     review_id|product_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+----------+-----------+-------------+-----------+----+-----------------+
| RZM83AGJAUZMA|B0142KIQYA|          1|           32|         41|   N|                Y|
|R12MYB9QM2YN8B|B00OLI95R6|          1|           19|         23|   N|                N|
| RGCY6WG37F810|B00ZN31MK6|          4|           24|         27|   N|                N|
|R2UH0VN09DDQLA|B0143LIXI2|          3|           23|         30|   N|                N|
|R1E4W9ROP10WT3|B013QMAZ80|          5|           19|         22|   N|                N|
| RF2BLXZTUOL0M|B013WF1Z4O|          5|           23|         27|   N|                N|
| REMDS9V4J58OR|B00X5FYC7Y|          4|           25|         29|   N|                Y|
|R34CAWTWR5O2VN|B0142KIQYA|          1|           18|         20|   N|                Y|
|R2ISFKK57JF02V|B00R6

In [16]:
# Determine the total number of all reviews
review_total = cleaned_df.count()
review_total

4056518

In [18]:
# Determine the total number of all 5-star reviews
total_fivestar = cleaned_df.filter(df["star_rating"] == 5).count()
total_fivestar

2445989

In [19]:
# Total number of paid reviews
paid_total = vine_paid.count()
paid_total

0

In [20]:
# Total number of unpaid reviews
unpaid_total = vine_not_paid.count()
unpaid_total

11362

In [21]:
# Total number of 5-star paid reviews
fivestar_paid = vine_paid.filter(vine_paid["star_rating"] == 5).count()
fivestar_paid

0

In [22]:
# Determine total number of 5-star unpaid reviews
fivestar_unpaid = vine_not_paid.filter(vine_paid["star_rating"] == 5).count()
fivestar_unpaid

4670

In [24]:
# Determine percentage of 5-star unpaid reviews
percentage_unpaid = (fivestar_unpaid / unpaid_total) * 100
percentage_unpaid

41.10191867628939

In [25]:
# Colab Featured Interactive Table
from google.colab import data_table 
data_table.enable_dataframe_formatter()

In [26]:
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12190288|R3FU16928EP5TC|B00AYB1482|     668895143|Enlightened: Seas...|Digital_Video_Dow...|          5|            0|          0|   N|                Y|I loved it and I ...|I loved it and I ...| 2015-08-31|
|         US|   30549954|R1IZHHS1MH3AQ4|B00KQD28OM|     246219280|             Vicious|Digital_Video_Dow

In [27]:
# No Nulls dropped
review_analysis2_df = df.select(["review_id", "product_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
votes20_df = review_analysis2_df.filter("total_votes >= 20")
votes20_df.show()

+--------------+----------+-----------+-------------+-----------+----+-----------------+
|     review_id|product_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+----------+-----------+-------------+-----------+----+-----------------+
|R1CY3APKBOVUYU|B00VO8D13K|          1|            9|         33|   N|                Y|
|R2LTI28AN2281A|B00YOSAJBO|          1|            0|         22|   N|                Y|
|R3PENXZQNYN29L|B00XOX9QCS|          1|            1|         26|   N|                Y|
| R8IGXNWIBN7ZP|B007BVMVDU|          1|            6|         20|   N|                Y|
|R2P8B83Q61L4HS|B00XOXDXV8|          1|            0|         22|   N|                Y|
| RZM83AGJAUZMA|B0142KIQYA|          1|           32|         41|   N|                Y|
|R257YW4XQLDH1M|B006W0QOF2|          1|            6|         20|   N|                Y|
|R12MYB9QM2YN8B|B00OLI95R6|          1|           19|         23|   N|                N|
| RGCY6WG37F810|B00ZN

In [28]:
helpful_df = votes20_df.filter(votes20_df["helpful_votes"]/votes20_df["total_votes"] >= 0.5)
helpful_df.show()

+--------------+----------+-----------+-------------+-----------+----+-----------------+
|     review_id|product_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+----------+-----------+-------------+-----------+----+-----------------+
| RZM83AGJAUZMA|B0142KIQYA|          1|           32|         41|   N|                Y|
|R12MYB9QM2YN8B|B00OLI95R6|          1|           19|         23|   N|                N|
| RGCY6WG37F810|B00ZN31MK6|          4|           24|         27|   N|                N|
|R2UH0VN09DDQLA|B0143LIXI2|          3|           23|         30|   N|                N|
|R1E4W9ROP10WT3|B013QMAZ80|          5|           19|         22|   N|                N|
| RF2BLXZTUOL0M|B013WF1Z4O|          5|           23|         27|   N|                N|
| REMDS9V4J58OR|B00X5FYC7Y|          4|           25|         29|   N|                Y|
|R34CAWTWR5O2VN|B0142KIQYA|          1|           18|         20|   N|                Y|
|R2ISFKK57JF02V|B00R6

In [29]:
# Filter rows where a review was written as part of the Vine paid program
vine_paid = helpful_df.filter(helpful_df["vine"] == "Y")
vine_paid.show()

# Filter rows where a review was written that was not part of the Vine paid program
vine_not_paid = helpful_df.filter(helpful_df["vine"] == "N")
vine_not_paid.show()

+---------+----------+-----------+-------------+-----------+----+-----------------+
|review_id|product_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+---------+----------+-----------+-------------+-----------+----+-----------------+
+---------+----------+-----------+-------------+-----------+----+-----------------+

+--------------+----------+-----------+-------------+-----------+----+-----------------+
|     review_id|product_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+----------+-----------+-------------+-----------+----+-----------------+
| RZM83AGJAUZMA|B0142KIQYA|          1|           32|         41|   N|                Y|
|R12MYB9QM2YN8B|B00OLI95R6|          1|           19|         23|   N|                N|
| RGCY6WG37F810|B00ZN31MK6|          4|           24|         27|   N|                N|
|R2UH0VN09DDQLA|B0143LIXI2|          3|           23|         30|   N|                N|
|R1E4W9ROP10WT3|B013QMAZ80|          5| 